# Task 1: Recurrent Neural Network Basics


## Recurrent Unit
Let's define the following recurrent unit
$$ h_t = f(W_{hh} h_{t-1} + W_{hx}x_t) $$
where $$t=1,\ldots,\infty,$$
$$h_t\in\mathbb{R}^{n\times 1} \text{ is the current hidden state}, $$
$$h_{t-1}\in\mathbb{R}^{n\times 1} \text{ is the previous hidden state}, $$
$$x_t \in\mathbb{R}^{d\times 1} \text{ is the current input}, $$
$$W_{hh}\in\mathbb{R}^{n\times n}, W_{hx} \in\mathbb{R}^{n\times d} \text{ are parameter matrices}.$$
Here, f(y) is the sigmoid function, $$f(y)=\frac{1}{1+\exp(-y)}.$$

What does the sigmoid function look like?

In [ ]:
## Plot sigmoid function in 1D
%matplotlib inline
import matplotlib
import numpy as np

import matplotlib.pyplot as plt

def sigmoid(y):
    return 1 / (1 + np.exp(-y))

# You can change the bound from 5, 10, 20 up to 50 to see the shape zoomed in/out
bound = 5
x = np.arange(-bound, bound, 0.5)
plt.plot(x, sigmoid(x))
plt.show()

In [ ]:
## How would you write matrix operations in code?

## Matrix-vector multiplication
## A is a n-by-d maxtrix, b is a dx1 vector
## c is a n-by-1 vector, the result of multiplying A and b
n = 3
d = 5
A = np.ones([n, d])   # initialize A as a n-by-d all-one matrix
print('The size of matrix A is {}-by-{}'.format(A.shape[0], A.shape[1]))
b = np.ones([d, 1]) # initialize b as a d-by-1 all-one vector
print('The size of vector b is {}-by-{}'.format(b.shape[0], b.shape[1]))
c = np.dot(A, b)
print('The size of vector c is {}-by-{}'.format(c.shape[0], c.shape[1]))
print('The vector c is \n', c)


### Forward Function
Let's code up the forward function of the recurrent unit.
Please fill in `< input your code here >` below.

In [ ]:
## Given input x, previous state hprev, parameter matrices Whx and Whh
## Compute the current state h
## Code up the forward function of the recurrent unit define above
## Remember to return the current state h

hidden_size = 10
input_size = 5

Whh = np.zeros([hidden_size, hidden_size])
Whh += np.random.uniform(-0.1, 0.1, [hidden_size, hidden_size])

Whx = np.zeros([hidden_size, input_size])
Whx += np.random.uniform(-0.1, 0.1, [hidden_size, input_size])

def forward_function(x, hprev, Whx, Whh):
    ## first compute the matrix-vector multiplication
    # Whx x + Whh hprev
    
    # TODO:
    h = <input your code here>

    ## use the sigmoid function to compute the current state
    # TODO
    h = sigmoid(<input your code here>)
    return h


x = np.random.randn(input_size, 1)
hprev = np.random.randn(hidden_size, 1)
h = forward_function(x, hprev, Whx, Whh)

print('Successfully forwarded!')    

In [ ]:
## test whether the forward function is correct
x_test = np.zeros([input_size, 1])
hprev_test = np.zeros([hidden_size, 1])
h_test = forward_function(x_test, hprev_test, Whx, Whh)
assert np.all(h_test - 0.5 < 1e-7)
print('Test passed!')

### Backward Function
An important component of finding the best parameters is to train the model by optimization. Let's consider a small, specific example: $ y = 2w^2 + 3$. 

If we want to *minimize* the value of $y$, what should our weight value $w$ be? We take the derivative of y with respect to w and solve for the value that makes this derivative 0:

$$ \frac{\partial y}{\partial w} = \frac{\partial (2w^2 + 3)}{\partial w} = 4w $$

Therefore, the $w$ that minimizes $y$ is $w=0$.

For neural networks, usually $y$ is some *cost* function that depends on your *parameter* $w$. Our objective is to find the $w$ that minimizes such cost function. In the recurrent neural network example we're looking at, the parameters we wish to find to minimize $y$ are the weight matrices $W_{hh}$ and $W_{hx}$. Since these are matrices rather than scalars, we need to do a bit of matrix calculus, but the key idea for optimization is the same!

#### Here are the important steps:

First let us define
$$ y = W_{hh} h_{t-1} + W_{hx} x_t.$$
Then
$$ h_t = f(y) $$
#### Important gradient \#1: 

$$ 
\frac{\partial h_t}{\partial h_{t-1}} = W_{hh}^T f\prime(y),
$$
where
$$
f\prime(y)=\frac{\partial f(y)}{\partial y}=\frac{\partial}{\partial y} [\frac{1}{1+\exp(-y)}]=\frac{\exp(-y)}{(1+\exp(-y))^2} = (1-f(y))f(y),
$$ is the gradient of sigmoid function.


Can you verfiy the above gradient? Why would you want to write the gradient in the last form?

#### Important gradient \#2:
$$ 
\frac{\partial h_t}{\partial W_{hh}} = \frac{\partial h_t}{\partial y} h_{t-1}^T = f\prime(y) h_{t-1}^T
$$

#### Important gradient \#3:
$$ 
\frac{\partial h_t}{\partial W_{hx}} = \frac{\partial h_t}{\partial y} x_{t}^T = f\prime(y) x_{t}^T
$$


Let's code up a backward function which accepts a training/error signal to weight the gradients and output the three gradients above. 
Please fill in `< input your code here >` below.

In [ ]:
## Given a training/error signal dEdh, input x, previous state hprev
## parameters Whh, Whx
## Return the three gradients above

def backward_function(x, hprev, dEdh, Whx, Whh):
    ## compute the gradient of sigmoid function
    f_prime = <input your code here>
    ## weight the gradient by training/error signal
    f_prime *= dEdh
    ## compute the gradient one 
    dEdhprev = <input your code here>
    ## compute the gradient two
    dWhh = <input your code here>
    ## compute the gradient three
    dWhx = <input your code here>
    return f_prime.T, dWhx, dWhh

# compute gradients
E = np.sum(h)
dEdh = np.ones([hidden_size, 1])

dEdhprev, dWhx, dWhh = backward_function(x, hprev, dEdh, Whx, Whh)

print('Successfully backpropgated!')

In [ ]:
## Check whether your code is right

# Numerical gradient computation
epsilon = 1e-7                                         
numdWhh = np.zeros([hidden_size,hidden_size])       
for i in range(hidden_size):                           
    for j in range(hidden_size):                       
        newWhh = np.copy(Whh)                          
        newWhh[i,j] += epsilon                         
                                                           
        h = forward_function(x, hprev, Whx, newWhh)
        newE = np.sum(h)                               
        numdWhh[i,j] = (newE - E) / epsilon            
                                                           
diff = np.sum(numdWhh - dWhh)   
print('Diff is ', diff)assert diff < 1e-3                                     
print('Test Passed!')

### Congratulations! You have learned the key part of a recurrent neural network.
